In [1]:
import os, glob, pandas as pd
import string, re
import numpy as np
from collections import Counter
import numbers
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from string import ascii_lowercase
import requests
import shutil
import pandas as pd
import time
from nltk import ngrams
from operator import itemgetter
import ast
import pickle

os.chdir('C:\\Users\\Ruben\\Documents\\GitHub\\dhl-ads')
from flashtext import KeywordProcessor
from functions_2 import hasNumbers
from functions_2 import ExtractQual
from functions_2 import GetNum
from functions_2 import NumberCandidateClass
from functions_2 import ExtractNum
from functions_2 import NonNumbClass
from functions_2 import NormalizeNumbers

In [2]:
os.chdir('C:\\Users\\Ruben\\Documents\\GitHub\\dhl-ads\\resources')
with open('qualitative_indicators.txt', encoding = 'utf-8') as f:
    qual_words = f.read().splitlines()

with open('wage_indicators.txt', encoding = 'utf-8') as f:
    wage_words = f.read().splitlines()
    
os.chdir('C:\\Users\\Ruben\\Documents\\GitHub\\dhl-ads\\evaluation\\txt')
list_annotations = glob.glob('*annotation.txt')

In [40]:
os.chdir('C:\\Users\\Ruben\\Documents\\GitHub\\dhl-ads\\data')
d = pd.read_csv('evaluation_sample_subsetted.csv')

In [41]:
d

,area,date,h,id,image_url,issue_id,max_x,max_y,min_x,min_y,n_pages,ocr,ocr_url,page,page_id,paper_title,w,clean,oc
0,5331732,1851/08/13 00:00:00,4005,ddd:010923676:mpeg21:a0013,http://imageviewer.kb.nl/ImagingService/imagin...,ddd:010923676:mpeg21,2635,4094,91,89,4,NADERE OPROEPING VAN SCHULDEISCHERS. De Schuld...,http://resolver.kb.nl/resolve?urn=ddd:01092367...,4,ddd:010923676:mpeg21:p004,Leydse courant,2544,nadere oproeping van schuldeischers de schulde...,['schoenmaker_13']
1,3899320,1851/01/28 00:00:00,2746,ddd:010583920:mpeg21:a0004,http://imageviewer.kb.nl/ImagingService/imagin...,ddd:010583920:mpeg21,2947,5155,1527,2409,4,STEEN HOUWER IJ. NHEERINGA Steenhouwer te Dock...,http://resolver.kb.nl/resolve?urn=ddd:01058392...,4,ddd:010583920:mpeg21:p004,Leeuwarder courant,1420,steen houwer ij nheeringa steenhouwer te docku...,"['houwer_1', 'steenhouwer_4', 'meesterknecht_4..."
2,3102094,1851/04/03 00:00:00,2867,ddd:010072874:mpeg21:a0015,http://imageviewer.kb.nl/ImagingService/imagin...,ddd:010072874:mpeg21,3315,2938,2233,71,6,OUDHEDEN -_ ZELDZAAIHEBEH. Op Dingsdag den 20s...,http://resolver.kb.nl/resolve?urn=ddd:01007287...,3,ddd:010072874:mpeg21:p003,Algemeen Handelsblad,1082,oudheden zeldzaaihebeh op dingsdag den 20sten ...,"['provisor_568', 'leerling_603', 'heelmeester_..."
3,2585050,1851/06/17 00:00:00,2665,ddd:010518350:mpeg21:a0016,http://imageviewer.kb.nl/ImagingService/imagin...,ddd:010518350:mpeg21,2976,4896,2006,2231,4,* * * Degenen die iets te vorderen hebben van ...,http://resolver.kb.nl/resolve?urn=ddd:01051835...,3,ddd:010518350:mpeg21:p003,Opregte Haarlemsche Courant,970,degenen die iets te vorderen hebben van of ver...,"['secondant_71', 'koekbakkersbediende_308', 'k..."
4,4590136,1851/12/11 00:00:00,5362,ddd:000020216:mpeg21:a0051,http://imageviewer.kb.nl/ImagingService/imagin...,ddd:000020216:mpeg21,4023,5413,2049,51,4,¦——¦¦w________ il i ¦mm —_w_^-»_nw—»—_——————¦»...,http://resolver.kb.nl/resolve?urn=ddd:00002021...,4,ddd:000020216:mpeg21:p004,"Nieuwe Rotterdamsche courant : staats-, handel...",1974,¦——¦¦w il i ¦mm —w»nw—»———————¦»—¦»¦¦———¦—m vo...,"['onderwijzer_472', 'schoolhouder_509']"
5,2426544,1851/09/14 00:00:00,2466,ddd:000020128:mpeg21:a0056,http://imageviewer.kb.nl/ImagingService/imagin...,ddd:000020128:mpeg21,3978,4486,2994,2020,4,I gebakgau's water. Ontvangen eene bezending v...,http://resolver.kb.nl/resolve?urn=ddd:00002012...,4,ddd:000020128:mpeg21:p004,"Nieuwe Rotterdamsche courant : staats-, handel...",984,i gebakgaus water ontvangen eene bezending ver...,['leerling_33']
6,2391778,1851/07/24 00:00:00,4811,ddd:010518374:mpeg21:a0020,http://imageviewer.kb.nl/ImagingService/imagin...,ddd:010518374:mpeg21,2972,4910,78,99,4,"Telt! De ""nd«rge«*ekende1 sedert eene reeks va...",http://resolver.kb.nl/resolve?urn=ddd:01051837...,3,ddd:010518374:mpeg21:p003,Opregte Haarlemsche Courant,2894,telt de nd«rge«ekende1 sedert eene reeks van j...,"['boekbinder_213', 'secondant_225', 'baas_322'..."
7,2193607,1851/10/02 00:00:00,2033,ddd:010276226:mpeg21:a0010,http://imageviewer.kb.nl/ImagingService/imagin...,ddd:010276226:mpeg21,3330,5016,2251,2983,2,HAARDEN EN KAGCHELS. Magazijnen: Langendelft B...,http://resolver.kb.nl/resolve?urn=ddd:01027622...,2,ddd:010276226:mpeg21:p002,Middelburgsche courant,1079,haarden en kagchels magazijnen langendelft b 1...,"['schipper_105', 'leerling_146', 'loopjongen_1..."
8,5333642,1851/04/11 00:00:00,3686,ddd:010583941:mpeg21:a0033,http://imageviewer.kb.nl/ImagingService/imagin...,ddd:010583941:mpeg21,1501,3798,54,112,10,LET WEL. «^. oteékende maakt door deze aan zij...,http://resolver.kb.nl/resolve?urn=ddd:01058394...,10,ddd:010583941:mpeg21:p010,Leeuwarder courant,1447,let wel « oteékende maakt door deze aan zijne ...,"['loodgieter_199', 'leidekker_201']"
9,5797941,1851/09/11 00:00:00,5251,ddd:010073023:mpeg21:a0025,http://imageviewer.kb.nl/ImagingService/imagin...,ddd:010073023:mpeg21,3337,5323,1173,72,4,Hf Alt UITVERKOOP VAN DE ENGELSCHE BAZAAR ONDE...,http://resolver.kb.n

In [3]:
# Create DF from annotations
df = pd.DataFrame([[int(l.split("_")[0]) for l in list_annotations]]).T
df['window'] = ''
df['annotated_occupation'] = ''
df['annotated_wage'] = ''
df['extracted_occupation'] = ''

for c,txt in enumerate(list_annotations):
    with open(txt, encoding = 'utf-8') as f:
        content = f.read().splitlines()
    df['window'][c] = content[2]
    df['annotated_occupation'][c] = content[0]
    df['annotated_wage'][c] = content[1]
    df['extracted_occupation'][c] = txt.split("_")[1]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykerne

In [4]:
## Run classifiers

df['extracted_qual'] = ""
df['extracted_quan'] = ""

for c,i in enumerate(df['window']):
    qual = ExtractQual(i, qual_words)
    quan = ExtractNum(i, qual_words)
    
    df['extracted_qual'][c] = qual
    df['extracted_quan'][c] = quan

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [5]:
df['annotated_qual'] = ""
df['annotated_quan'] = ""

for c,aw in enumerate(df['annotated_wage']):
    
    if aw == "na" or aw == "np":
        df['annotated_qual'][c] = aw
        df['annotated_quan'][c] = aw
        continue
    
    if aw != "na" and aw != "np":
        if "qual" in aw and "quan" in aw:
            aw = aw.split(',')
            df['annotated_quan'][c] = " ".join(aw[0].split(' ')[1:]) 
            df['annotated_qual'][c] = " ".join(aw[1].split(' ')[2:]) 

        if "qual" in aw and "quan" not in aw:
            aw = aw.split(' ')
            df['annotated_qual'][c] = " ".join(aw[1:]) 
            df['annotated_quan'][c] = "np"

        if "quan" in aw and "qual" not in aw:
            aw = aw.split(' ')
            df['annotated_qual'][c] = "np"
            df['annotated_quan'][c] = " ".join(aw[1:]) 
        
        

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [6]:
df

,0,window,annotated_occupation,annotated_wage,extracted_occupation,extracted_qual,extracted_quan,annotated_qual,annotated_quan
0,100,middelbaren leeftijd zag zich gaarne ten spoed...,na,na,huishoudster,None,na,na,na
1,101,gaarne dadelijk of later geplaatst te worden a...,jufvrouw huishoudster,np,huishoudster,None,na,np,np
2,102,zag zich gaarne ten spoedigste geplaatst in ee...,huishoudster kamenier,np,kamenier,None,na,np,np
3,103,middelbaren leeftijd zag zich gaarne ten spoed...,huishoudster kamenier,np,huishoudster,None,na,np,np
4,104,gaarne dadelijk of later geplaatst te worden a...,jufvrouw huishoudster,np,huishoudster,None,na,np,np
5,105,behoorende arm de markte van assendorp in zood...,na,na,landbouwer,hoog loon,na,na,na
6,106,eruideniersbedÃ¯ende met iÂ° mei wordt in eene...,﻿bediende,qual naar bekwaamheid,bediende,naar bekwaamheid,na,naar bekwaamheid,np
7,107,ining en salaris naar bekwaamheid adres franco...,na,na,kantoorboekhandelaar,naar bekwaamheid,na,na,na
8,108,9302 bij eene familie in de omstreken van haar...,﻿keukenmeid,quan Æ’l5o,keukenmeid,None,Æ’l5o,np,Æ’l5o
9,109,8 mei 1866 u Â« iwuiu Â«ui wÃ¶rÃ¯teni3poedigst...,molenmakersknecht,qual nader te bepalen,molenmakersknecht,nader te bepalen,na,nader te bepalen,np


In [8]:
df.fillna(value="na", inplace=True)

In [9]:
df = df.replace("np","na")

In [27]:
df2 = df[(df['annotated_occupation'] != "na")]
df2 =df2.reset_index(drop=True)

df3 = df[(df['annotated_wage'] != "na")]
df3 =df3.reset_index(drop=True)

In [36]:
def GetEvaluationCounts(df):
    evaldict = {
    "correct":0,
    "incorrect":0,
    "partial":0,
    "spurious":0,
    "missing":0
    }

    extractions = []

    for c in range(len(df)-1):

        for ner_type in ['quan','qual']:

            annotation = df['annotated_' + ner_type][c]
            extraction = df['extracted_' + ner_type][c]
            
            #if annotation == "na" and extraction == "na":
            #    continue

            if annotation == extraction:   
                evaldict['correct'] += 1
                result='correct'

            if annotation != extraction and extraction in annotation or extraction in annotation.split(' ') and extraction != "na":
                evaldict['partial'] += 1
                result='partial'

            if annotation == 'na' and extraction != 'na':
                evaldict['spurious'] += 1
                result='spurious'

            if annotation != 'na' and extraction == 'na':
                evaldict['missing'] += 1
                result='missing'

            if annotation != extraction and extraction not in annotation and extraction not in annotation.split(' '):
                evaldict['incorrect'] += 1
                result='incorrect'

            extractions.append([c,annotation,result])
    
    return [evaldict,extractions]

def GetEvaluationCountsNoFN(df):
    evaldict = {
    "correct":0,
    "incorrect":0,
    "partial":0,
    "spurious":0,
    "missing":0
    }

    extractions = []

    for c in range(len(df)-1):

        for ner_type in ['quan','qual']:

            annotation = df['annotated_' + ner_type][c]
            extraction = df['extracted_' + ner_type][c]
            
            if annotation == "na" and extraction == "na":
                continue

            if annotation == extraction:   
                evaldict['correct'] += 1
                result='correct'

            if annotation != extraction and extraction in annotation or extraction in annotation.split(' ') and extraction != "na":
                evaldict['partial'] += 1
                result='partial'

            if annotation == 'na' and extraction != 'na':
                evaldict['spurious'] += 1
                result='spurious'

            if annotation != 'na' and extraction == 'na':
                evaldict['missing'] += 1
                result='missing'

            if annotation != extraction and extraction not in annotation and extraction not in annotation.split(' '):
                evaldict['incorrect'] += 1
                result='incorrect'

            extractions.append([c,annotation,result])
    
    return [evaldict,extractions]

def GetPRF(evaldict):
    pos = evaldict['correct'] + evaldict['incorrect'] + evaldict['partial'] + evaldict['missing']
    act = evaldict['correct'] + evaldict['incorrect'] + evaldict['partial'] + evaldict['spurious']
    precision = (evaldict['correct'] + 0.5 * evaldict['partial']) / act
    recall = (evaldict['correct'] + 0.5 * evaldict['partial']) / pos
    f1 = 2 * (precision * recall) / (precision + recall)
    print("precision: {}".format(precision))
    print("recall: {}".format(recall))
    print("f-score: {}".format(f1))
    return [precision,recall,f1]

In [33]:
ed = GetEvaluationCounts(df)
res = GetPRF(ed[0])
ed[0]

precision: 0.7154882154882155
recall: 0.7535460992907801
f-score: 0.7340241796200345


{'correct': 408, 'incorrect': 96, 'partial': 34, 'spurious': 56, 'missing': 26}

In [34]:
ed2 = GetEvaluationCounts(df2)
res2 = GetPRF(ed2[0])
ed2[0]

precision: 0.7167381974248928
recall: 0.7373068432671082
f-score: 0.7268770402611534


{'correct': 317, 'incorrect': 77, 'partial': 34, 'spurious': 38, 'missing': 25}

In [35]:
ed3 = GetEvaluationCounts(df3)
res3 = GetPRF(ed3[0])
ed3[0]

precision: 0.6413043478260869
recall: 0.6069958847736625
f-score: 0.6236786469344608


{'correct': 131, 'incorrect': 53, 'partial': 33, 'spurious': 13, 'missing': 26}

In [39]:
ed4 = GetEvaluationCountsNoFN(df)
res4 = GetPRF(ed4[0])
ed4[0]

precision: 0.3130081300813008
recall: 0.35648148148148145
f-score: 0.3333333333333333


{'correct': 60, 'incorrect': 96, 'partial': 34, 'spurious': 56, 'missing': 26}